In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, precision_score, recall_score, auc
from sklearn.model_selection import KFold
torch.manual_seed(2)    # reproducible torch:2 np:3
np.random.seed(3)

from config import BIN_config_DBPE
from models import BIN_Interaction_Flat
from stream import BIN_Data_Encoder

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [2]:
def test(data_generator, model):
    y_pred = []
    y_label = []
    model.eval()
    loss_accumulate = 0.0
    count = 0.0
    for i, (d, p, d_mask, p_mask, label) in enumerate(data_generator):
        score, _ = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())
        
        m = torch.nn.Sigmoid()
        logits = torch.squeeze(m(score))
        loss_fct = torch.nn.BCELoss()            
        
        label = Variable(torch.from_numpy(np.array(label)).float()).cuda()

        loss = loss_fct(logits, label)
        
        loss_accumulate += loss
        count += 1
        
        logits = logits.detach().cpu().numpy()
        
        label_ids = label.to('cpu').numpy()
        y_label = y_label + label_ids.flatten().tolist()
        y_pred = y_pred + logits.flatten().tolist()
        
    loss = loss_accumulate/count
    
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)

    precision = tpr / (tpr + fpr)

    f1 = 2 * precision * tpr / (tpr + precision + 0.00001)

    thred_optim = thresholds[5:][np.argmax(f1[5:])]

    print("optimal threshold: " + str(thred_optim))

    y_pred_s = [1 if i else 0 for i in (y_pred >= thred_optim)]

    auc_k = auc(fpr, tpr)
    print("AUROC:" + str(auc_k))
    print("AUPRC: "+ str(average_precision_score(y_label, y_pred)))

    cm1 = confusion_matrix(y_label, y_pred_s)
    print('Confusion Matrix : \n', cm1)
    print('Recall : ', recall_score(y_label, y_pred_s))
    print('Precision : ', precision_score(y_label, y_pred_s))

    total1=sum(sum(cm1))
    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    print('Sensitivity : ', sensitivity1 )

    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)

    outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
    return roc_auc_score(y_label, y_pred), average_precision_score(y_label, y_pred), f1_score(y_label, outputs), y_pred, loss.item()


def main(fold_n, lr):
    config = BIN_config_DBPE()
    
    lr = lr
    BATCH_SIZE = config['batch_size']
    train_epoch = 100
    
    loss_history = []
    
    model = BIN_Interaction_Flat(**config)
    
    model = model.cuda()

    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model, dim = 0)
            
    opt = torch.optim.Adam(model.parameters(), lr = lr)
    #opt = torch.optim.SGD(model.parameters(), lr = lr, momentum=0.9)
    
    print('--- Data Preparation ---')
    
    params = {'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 6, 
              'drop_last': True}

    dataFolder = './dataset/BindingDB'
    df_train = pd.read_csv(dataFolder + '/train.csv')
    df_val = pd.read_csv(dataFolder + '/val.csv')
    df_test = pd.read_csv(dataFolder + '/test.csv')
    
    training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
    training_generator = data.DataLoader(training_set, **params)

    validation_set = BIN_Data_Encoder(df_val.index.values, df_val.Label.values, df_val)
    validation_generator = data.DataLoader(validation_set, **params)
    
    testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
    testing_generator = data.DataLoader(testing_set, **params)
    
    # early stopping
    max_auc = 0
    model_max = copy.deepcopy(model)
    
    print('--- Go for Training ---')
    torch.backends.cudnn.benchmark = True
    for epo in range(train_epoch):
        model.train()
        for i, (d, p, d_mask, p_mask, label) in enumerate(training_generator):
            score, _ = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())

            label = Variable(torch.from_numpy(np.array(label)).float()).cuda()
            
            loss_fct = torch.nn.BCELoss()
            m = torch.nn.Sigmoid()
            n = torch.squeeze(m(score))
            
            loss = loss_fct(n, label)
            loss_history.append(loss)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            if (i % 100 == 0):
                print('Training at Epoch ' + str(epo + 1) + ' iteration ' + str(i) + ' with loss ' + str(loss.cpu().detach().numpy()))
            
        # every epoch test
        with torch.set_grad_enabled(False):
            auc, auprc, f1, logits, loss = test(validation_generator, model)
            if auc > max_auc:
                model_max = copy.deepcopy(model)
                max_auc = auc
            
            print('Validation at Epoch '+ str(epo + 1) + ' , AUROC: '+ str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: '+str(f1))
    
    print('--- Go for Testing ---')
    try:
        with torch.set_grad_enabled(False):
            auc, auprc, f1, logits, loss = test(testing_generator, model_max)
            print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: '+str(f1) + ' , Test loss: '+str(loss))
    except:
        print('testing failed')
    return model_max, loss_history

In [ ]:
# fold 1
#biosnap interaction times 1e-6, flat, batch size 64, len 205, channel 3, epoch 50
s = time()
model_max, loss_history = main(1, 5e-6)
e = time()
print(e-s)
lh = list(filter(lambda x: x < 1, loss_history))
plt.plot(lh)

Let's use 8 GPUs!
--- Data Preparation ---
--- Go for Training ---


/home/ken/miniconda3/envs/MolTrans/lib/python3.7/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


Training at Epoch 1 iteration 0 with loss 0.67082864
Training at Epoch 1 iteration 100 with loss 0.6956154
Training at Epoch 1 iteration 200 with loss 0.6720772
Training at Epoch 1 iteration 300 with loss 0.70531034
Training at Epoch 1 iteration 400 with loss 0.6877631
Training at Epoch 1 iteration 500 with loss 0.71080685
Training at Epoch 1 iteration 600 with loss 0.6649274
Training at Epoch 1 iteration 700 with loss 0.69906
optimal threshold: 0.023896418511867523
AUROC:0.5510503926924208
AUPRC: 0.1653136126373162
Confusion Matrix : 
 [[   2 5712]
 [   0  926]]
Recall :  1.0
Precision :  0.13949984935221452
Accuracy :  0.13975903614457832
Sensitivity :  0.00035001750087504374
Specificity :  1.0
Validation at Epoch 1 , AUROC: 0.5510503926924208 , AUPRC: 0.1653136126373162 , F1: 0.2431283649759139


/home/ken/miniconda3/envs/MolTrans/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


Training at Epoch 2 iteration 0 with loss 0.70074856
Training at Epoch 2 iteration 100 with loss 0.7485028
Training at Epoch 2 iteration 200 with loss 0.7008172
Training at Epoch 2 iteration 300 with loss 0.6781282
Training at Epoch 2 iteration 400 with loss 0.69029725
Training at Epoch 2 iteration 500 with loss 0.71803224
Training at Epoch 2 iteration 600 with loss 0.66221803
Training at Epoch 2 iteration 700 with loss 0.7087128
optimal threshold: 1.599509715788372e-07
AUROC:0.5901429250728195
AUPRC: 0.1913381390805916
Confusion Matrix : 
 [[   5 5709]
 [   0  926]]
Recall :  1.0
Precision :  0.13956292388847025
Accuracy :  0.140210843373494
Sensitivity :  0.0008750437521876093
Specificity :  1.0
Validation at Epoch 2 , AUROC: 0.5901429250728195 , AUPRC: 0.1913381390805916 , F1: 0.271850170261067


...

Training at Epoch 23 iteration 0 with loss 0.668189
Training at Epoch 23 iteration 100 with loss 0.6526967
Training at Epoch 23 iteration 200 with loss 0.690978
Training at Epoch 23 iteration 300 with loss 0.7117002
Training at Epoch 23 iteration 400 with loss 0.65555227
Training at Epoch 23 iteration 500 with loss 0.6775181
Training at Epoch 23 iteration 600 with loss 0.65972304
Training at Epoch 23 iteration 700 with loss 0.6382162
optimal threshold: 0.42399075627326965
AUROC:0.646658088843967
AUPRC: 0.20183212796506594
Confusion Matrix : 
 [[1352 4362]
 [  87  839]]
Recall :  0.906047516198704
Precision :  0.16131513170544126
Accuracy :  0.3299698795180723
Sensitivity :  0.23661183059152957
Specificity :  0.906047516198704
Validation at Epoch 23 , AUROC: 0.646658088843967 , AUPRC: 0.20183212796506594 , F1: 0.30818791946308727


/home/ken/miniconda3/envs/MolTrans/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


Training at Epoch 24 iteration 0 with loss 0.68990356
Training at Epoch 24 iteration 100 with loss 0.7046473
Training at Epoch 24 iteration 200 with loss 0.7651698
Training at Epoch 24 iteration 300 with loss 0.6845608
Training at Epoch 24 iteration 400 with loss 0.6877582
Training at Epoch 24 iteration 500 with loss 0.6195511
Training at Epoch 24 iteration 600 with loss 0.6992395
Training at Epoch 24 iteration 700 with loss 0.6584714
optimal threshold: 0.39975234866142273
AUROC:0.6710561585207255
AUPRC: 0.25076450096947317
Confusion Matrix : 
 [[1771 3944]
 [ 124  801]]
Recall :  0.865945945945946
Precision :  0.16880927291886197
Accuracy :  0.38734939759036147
Sensitivity :  0.30988626421697285
Specificity :  0.865945945945946
Validation at Epoch 24 , AUROC: 0.6710561585207255 , AUPRC: 0.25076450096947317 , F1: 0.3278563240351548


/home/ken/miniconda3/envs/MolTrans/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


Training at Epoch 25 iteration 0 with loss 0.63341814
Training at Epoch 25 iteration 100 with loss 0.68980896
Training at Epoch 25 iteration 200 with loss 0.65841216
Training at Epoch 25 iteration 300 with loss 0.7085751
Training at Epoch 25 iteration 400 with loss 0.71747935
Training at Epoch 25 iteration 500 with loss 0.6988963
Training at Epoch 25 iteration 600 with loss 0.7091571
Training at Epoch 25 iteration 700 with loss 0.6682226
